In [1]:
import cv2
import dlib
import time
import threading
import math
import numpy as np

from IPython.display import display, Image, clear_output
from io import BytesIO
import PIL
%matplotlib inline

In [2]:
!wget https://raw.githubusercontent.com/chuanqi305/MobileNet-SSD/master/voc/MobileNetSSD_deploy.prototxt

--2023-10-09 07:01:39--  https://raw.githubusercontent.com/chuanqi305/MobileNet-SSD/master/voc/MobileNetSSD_deploy.prototxt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29353 (29K) [text/plain]
Saving to: ‘MobileNetSSD_deploy.prototxt.4’

MobileNetSSD_deploy 100%[===================>]  28.67K  --.-KB/s    in 0.005s  

2023-10-09 07:01:39 (5.28 MB/s) - ‘MobileNetSSD_deploy.prototxt.4’ saved [29353/29353]



In [3]:
!wget https://github.com/PINTO0309/MobileNet-SSD-RealSense/raw/master/caffemodel/MobileNetSSD/MobileNetSSD_deploy.caffemodel

--2023-10-09 07:01:39--  https://github.com/PINTO0309/MobileNet-SSD-RealSense/raw/master/caffemodel/MobileNetSSD/MobileNetSSD_deploy.caffemodel
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/PINTO0309/MobileNet-SSD-RealSense/master/caffemodel/MobileNetSSD/MobileNetSSD_deploy.caffemodel [following]
--2023-10-09 07:01:40--  https://raw.githubusercontent.com/PINTO0309/MobileNet-SSD-RealSense/master/caffemodel/MobileNetSSD/MobileNetSSD_deploy.caffemodel
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23147564 (22M) [application/octet-stream]
Saving to: ‘MobileNetSSD_deploy.caffemodel.4’

MobileNetS

In [4]:
video = cv2.VideoCapture("fast-forklift-operator-360-ytshorts.savetube.me.mp4")

In [5]:
ssd = cv2.dnn.readNetFromCaffe("./MobileNetSSD_deploy.prototxt","./MobileNetSSD_deploy.caffemodel")

In [6]:
def estimateSpeed(location1, location2,ppm=8.8,fps=24):
    d_pixels = math.sqrt(math.pow(location2[0] - location1[0], 2) + math.pow(location2[1] - location1[1], 2))
    d_meters = d_pixels / ppm
    fps = 18
    speed = d_meters * fps * 3.6
    return speed

In [ ]:
def trackMultipleObjects(WIDTH = 1280, HEIGHT = 720):
    rectangleColor = (0, 255, 0)
    frameCounter = 0
    currentCarID = 0
    fps = 0

    carTracker = {}
    carNumbers = {}
    carLocation1 = {}
    carLocation2 = {}
    tracker_info = {}
    speed = [None] * 1000
    
    while True:
        start_time = time.time()
        rc, image = video.read()
        if type(image) == type(None):
            break

        image = cv2.resize(image, (WIDTH, HEIGHT))
        resultImage = image.copy()

        frameCounter = frameCounter + 1

        carIDtoDelete = []

        for carID in carTracker.keys():
            trackingQuality = carTracker[carID].update(image)

            if trackingQuality < 7:
                carIDtoDelete.append(carID)

        for carID in carIDtoDelete:
            print ('Removing carID ' + str(carID) + ' from list of trackers.')
            carTracker.pop(carID, None)
            carLocation1.pop(carID, None)
            carLocation2.pop(carID, None)

        if not (frameCounter % 10):
            blob = cv2.dnn.blobFromImage(cv2.resize(image,(300,300)),0.007843,(400,400),(127.5,127.5,127.5),False)
            ssd.setInput(blob)
            detection_ssd = ssd.forward()
            cars=[]

            for i in range(detection_ssd.shape[2]):
                confidence=detection_ssd[0,0,i,2]
                if confidence > 0.2:
                    class_id = int(detection_ssd[0,0,i,1])
                    if class_id in [6,7]:
                        xLeftBottom = int(detection_ssd[0, 0, i, 3] * 300) 
                        yLeftBottom = int(detection_ssd[0, 0, i, 4] * 300)
                        xRightTop   = int(detection_ssd[0, 0, i, 5] * 300)
                        yRightTop   = int(detection_ssd[0, 0, i, 6] * 300)

                        heightFactor = image.shape[0]/300.0  
                        widthFactor = image.shape[1]/300.0 

                        xLeftBottom = int(widthFactor * xLeftBottom) 
                        yLeftBottom = int(heightFactor * yLeftBottom)
                        xRightTop   = int(widthFactor * xRightTop)
                        yRightTop   = int(heightFactor * yRightTop)
                        class_name = 'car' if class_id == 7 else 'bus'

                        cars.append([xLeftBottom,yLeftBottom,xRightTop-xLeftBottom,yRightTop-yLeftBottom,confidence,class_name])  

            for (_x, _y, _w, _h, _c, _n) in cars:
                x = int(_x)
                y = int(_y)
                w = int(_w)
                h = int(_h)
                conf = str('%.3f' %_c)
                det_class = str(_n)

                x_bar = x + 0.5 * w
                y_bar = y + 0.5 * h
                
                matchCarID = None

                for carID in carTracker.keys():
                    trackedPosition = carTracker[carID].get_position()
                        
                    t_x = int(trackedPosition.left())
                    t_y = int(trackedPosition.top())
                    t_w = int(trackedPosition.width())
                    t_h = int(trackedPosition.height())
                        
                    t_x_bar = t_x + 0.5 * t_w
                    t_y_bar = t_y + 0.5 * t_h
                    
                    if ((t_x <= x_bar <= (t_x + t_w)) and (t_y <= y_bar <= (t_y + t_h)) and (x <= t_x_bar <= (x + w)) and (y <= t_y_bar <= (y + h))):
                        matchCarID = carID
                    
                if matchCarID is None:
                    print ('Creating new tracker ' + str(currentCarID))
                        
                    tracker = dlib.correlation_tracker()
                    tracker.start_track(image, dlib.rectangle(x, y, x + w, y + h))
                        
                    carTracker[currentCarID] = tracker
                    carLocation1[currentCarID] = [x, y, w, h]
                    tracker_info[currentCarID] = [det_class,conf]
                    currentCarID = currentCarID + 1


        for carID in carTracker.keys():
            trackedPosition = carTracker[carID].get_position()
                    
            t_x = int(trackedPosition.left())
            t_y = int(trackedPosition.top())
            t_w = int(trackedPosition.width())
            t_h = int(trackedPosition.height())
                
            cv2.rectangle(resultImage, (t_x, t_y), (t_x + t_w, t_y + t_h), rectangleColor, 4)
                
            carLocation2[carID] = [t_x, t_y, t_w, t_h]
            
        end_time = time.time()
            
        if not (end_time == start_time):
            fps = 1.0/(end_time - start_time)
            
        cv2.putText(resultImage, 'FPS: ' + str(int(fps)), (620, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)


        for i in carLocation1.keys():
            if frameCounter % 1 == 0:
                [x1, y1, w1, h1] = carLocation1[i]
                [x2, y2, w2, h2] = carLocation2[i]

                carLocation1[i] = [x2, y2, w2, h2]

                if [x1, y1, w1, h1] != [x2, y2, w2, h2]:
                    if (speed[i] == None or speed[i] == 0) and y1 >= 275 and y1 <= 285:
                        speed[i] = estimateSpeed([x1, y1, w1, h1], [x2, y2, w2, h2])

                    if speed[i] != None and y1 >= 180 and h1 > 100 and h2 > 100:
                        cv2.putText(resultImage, tracker_info[i][0] +"("+tracker_info[i][1]+")" + str(i) + ": "+str(int(speed[i])) + " km/hr", (int(x1 + w1/2), int(y1-5)),cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)

        frame=cv2.cvtColor(resultImage, cv2.COLOR_BGR2RGB) 
        f = BytesIO()
        PIL.Image.fromarray(frame).save(f,'jpeg')
        clear_output(wait=True)
        display(Image(f.getvalue()))

trackMultipleObjects()
